In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

In [2]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import roc_auc_score
# from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


from sklearn.metrics import classification_report, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE

In [3]:
train_ops = pd.read_csv('data/train/train_flag.csv')
test_ops = pd.read_csv('data/test/test_flag.csv')
# acc_data = pd.read_json('data/train/accounts_data_train.json')
# enq_data = pd.read_json('data/train/enquiry_data_train.json')

In [4]:
# loading all training data

with open('data/train/accounts_data_train.json', 'r') as f:
	accounts = json.loads(f.read())

accounts = [item for data in accounts for item in data]
acc_data = pd.DataFrame(accounts)

with open('data/train/enquiry_data_train.json', 'r') as f:
	enquiry = json.loads(f.read())

enquiry = [item for data in enquiry for item in data]
enq_data = pd.DataFrame(enquiry)


In [5]:
# loading all testing data

with open('data/test/accounts_data_test.json', 'r') as f:
	accounts_t = json.loads(f.read())

accounts_t = [item for data in accounts_t for item in data]
acc_data_t = pd.DataFrame(accounts_t)

with open('data/test/enquiry_data_test.json', 'r') as f:
	enquiry_t = json.loads(f.read())

enquiry_t = [item for data in enquiry_t for item in data]
enq_data_t = pd.DataFrame(enquiry_t)

In [5]:
# split payment history into delayed day values

def payment_string_split(x):
	return [int(x[i:i+3]) for i in range(0, len(x), 3)]

acc_data['payment_hist_string'] = acc_data['payment_hist_string'].apply(payment_string_split)
acc_data_t['payment_hist_string'] = acc_data_t['payment_hist_string'].apply(payment_string_split)
acc_data.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550
2,Credit card,80996.445,0.0,2020-06-29,NaN,"[0, 0, 0, 0, 0, 0]",AAA10545297
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888
4,Credit card,10480.500,0.0,2014-09-10,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915


In [ ]:
# use the delayed days values to fill empties in close_date

today_date = pd.Timestamp.today().date()

def close_date_filler(item):
	if(pd.isna(item['closed_date'])):
		delta = len(item['payment_hist_string'])
		current_date = pd.to_datetime(item['open_date'])
		overdue_flag = item['amount_overdue']!=0
		if(overdue_flag):
			item['closed_date'] = str(today_date)
		else:
			item['closed_date'] = str((current_date + pd.DateOffset(months=delta)).date())
	return item['closed_date']
	

In [ ]:
# z = acc_data.iloc[461]
# z = close_date_filler(z)
# z, len(z['payment_hist_string'])

acc_data['closed_date'] = acc_data[['closed_date', 'open_date', 'payment_hist_string', 'amount_overdue']].apply(close_date_filler, axis=1)
acc_data_t['closed_date'] = acc_data_t[['closed_date', 'open_date', 'payment_hist_string', 'amount_overdue']].apply(close_date_filler, axis=1)
acc_data.head(10)

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915
5,Consumer credit,46696.500,0.0,2016-11-20,2017-10-06,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",AAA20326915
6,Credit card,0.000,0.0,2014-09-10,2018-08-30,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915
7,Consumer credit,117000.000,0.0,2018-02-04,2018-05-06,"[0, 0, 0]",AAA20326915
8,Consumer credit,90000.000,0.0,2020-03-12,2020-04-12,[0],AAA20326915
9,Consumer credit,235800.000,0.0,2020-03-29,2020-05-13,[0],AAA20326915


In [ ]:
acc_data.isna().sum()

credit_type            0
loan_amount            3
amount_overdue         0
open_date              0
closed_date            0
payment_hist_string    0
uid                    0
dtype: int64

In [ ]:
acc_data = acc_data.dropna(axis=0)
acc_data.isna().sum()

credit_type            0
loan_amount            0
amount_overdue         0
open_date              0
closed_date            0
payment_hist_string    0
uid                    0
dtype: int64

In [6]:
acc_data = pd.read_csv('account_data_cleaned.csv')
acc_data = acc_data.drop(columns=['Unnamed: 0'], axis=0)

acc_data_t = pd.read_csv('account_data_cleaned_t.csv')
acc_data_t = acc_data_t.drop(columns=['Unnamed: 0'], axis=0)

acc_data.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915


In [7]:
acc_data['payment_hist_string'] = acc_data['payment_hist_string'].apply(lambda x: eval(x))
acc_data_t['payment_hist_string'] = acc_data_t['payment_hist_string'].apply(lambda x: eval(x))

In [8]:
acc_data[['open_date', 'closed_date']] = acc_data[['open_date', 'closed_date']].apply(pd.to_datetime)
acc_data_t[['open_date', 'closed_date']] = acc_data_t[['open_date', 'closed_date']].apply(pd.to_datetime)
acc_data.dtypes

credit_type                    object
loan_amount                   float64
amount_overdue                float64
open_date              datetime64[ns]
closed_date            datetime64[ns]
payment_hist_string            object
uid                            object
dtype: object

<h1>Feature Engineering</h1>

<h3>Account - Date Based Features</h3>

In [9]:
def extract_date_features(df, column="date"):
	# date = x.day
	# day = x.day_of_week+1
	# month = x.month
	# is_weekday = int(1<=day<=5)
	# quarter = x.quarter
	# is_quarter_start = x.is_quarter_start
	# is_quarter_end = x.is_quarter_end
	# year = x.year

	# return pd.Series([day, date, month, year, quarter, is_weekday])

	# df[column] = pd.to_datetime(df[column])  # Ensure datetime
	# df['date_str'] = df[column].dt.strftime('%Y-%m-%d')
	pre = column[0]
	df[f'{pre}_date'] = df[column].dt.day
	df[f'{pre}_day'] = df[column].dt.day_of_week+1
	df[f'{pre}_month'] = df[column].dt.month
	df[f'{pre}_year'] = df[column].dt.year
	df[f'{pre}_quarter'] = df[column].dt.quarter
	df[f'{pre}_is_weekday'] = df[column].dt.dayofweek < 5

	return df


# extract_date_features(acc_data.iloc[:10], column='open_date')

In [10]:
user_df = acc_data.copy()
user_df_t = acc_data_t.copy()

# df = user_df.iloc[:10]
# df[['o_day', 'o_date', 'o_month', 'o_year', 'o_quarter', 'o_is_weekday']] = df['open_date'].apply(extract_date_features)

# user_df[['o_day', 'o_date', 'o_month', 'o_year', 'o_quarter', 'o_is_weekday']] = user_df['open_date'].apply(extract_date_features)
# user_df[['c_day', 'c_date', 'c_month', 'c_year', 'c_quarter', 'c_is_weekday']] = user_df['closed_date'].apply(extract_date_features)

user_df = extract_date_features(user_df, column='open_date')
user_df = extract_date_features(user_df, column='closed_date')

user_df_t = extract_date_features(user_df_t, column='open_date')
user_df_t = extract_date_features(user_df_t, column='closed_date')

user_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,o_date,o_day,o_month,o_year,o_quarter,o_is_weekday,c_date,c_day,c_month,c_year,c_quarter,c_is_weekday
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550,22,6,9,2018,3,False,22,6,2,2020,1,False
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550,8,4,3,2018,1,True,25,4,7,2019,3,True
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297,29,1,6,2020,2,True,29,2,12,2020,4,True
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888,9,2,6,2020,2,True,9,3,9,2020,3,True
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915,10,3,9,2014,3,True,10,7,1,2021,1,False


In [11]:
user_df['loan_duration_m'] = round((user_df['closed_date'] - user_df['open_date']) / pd.Timedelta(30.41, 'D'))
user_df_t['loan_duration_m'] = round((user_df_t['closed_date'] - user_df_t['open_date']) / pd.Timedelta(30.41, 'D'))
# loan_dur.head(), user_df[['closed_date', 'open_date']].head()
user_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,o_date,o_day,o_month,o_year,o_quarter,o_is_weekday,c_date,c_day,c_month,c_year,c_quarter,c_is_weekday,loan_duration_m
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550,22,6,9,2018,3,False,22,6,2,2020,1,False,17.0
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550,8,4,3,2018,1,True,25,4,7,2019,3,True,17.0
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297,29,1,6,2020,2,True,29,2,12,2020,4,True,6.0
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888,9,2,6,2020,2,True,9,3,9,2020,3,True,3.0
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915,10,3,9,2014,3,True,10,7,1,2021,1,False,76.0


<h3>Account - Numerical Data based Features</h3>

In [12]:
user_df['max_delay'] = user_df['payment_hist_string'].apply(lambda x: max(x) if len(x)>0 else 0)
user_df['avg_delay'] = user_df['payment_hist_string'].apply(lambda x: np.mean(x) if len(x)>0 else 0)
user_df_t['max_delay'] = user_df_t['payment_hist_string'].apply(lambda x: max(x) if len(x)>0 else 0)
user_df_t['avg_delay'] = user_df_t['payment_hist_string'].apply(lambda x: np.mean(x) if len(x)>0 else 0)
user_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,o_date,o_day,o_month,...,o_is_weekday,c_date,c_day,c_month,c_year,c_quarter,c_is_weekday,loan_duration_m,max_delay,avg_delay
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550,22,6,9,...,False,22,6,2,2020,1,False,17.0,10,0.588235
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550,8,4,3,...,True,25,4,7,2019,3,True,17.0,44,3.625000
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297,29,1,6,...,True,29,2,12,2020,4,True,6.0,0,0.000000
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888,9,2,6,...,True,9,3,9,2020,3,True,3.0,0,0.000000
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915,10,3,9,...,True,10,7,1,2021,1,False,76.0,917,190.486842


In [13]:
user_df['is_loan_active'] = user_df['amount_overdue']>0
user_df_t['is_loan_active'] = user_df_t['amount_overdue']>0

In [14]:
user_df[user_df.select_dtypes(bool).columns] = user_df.select_dtypes(bool).astype(int)
user_df_t[user_df_t.select_dtypes(bool).columns] = user_df_t.select_dtypes(bool).astype(int)
user_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,o_date,o_day,o_month,...,c_date,c_day,c_month,c_year,c_quarter,c_is_weekday,loan_duration_m,max_delay,avg_delay,is_loan_active
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550,22,6,9,...,22,6,2,2020,1,0,17.0,10,0.588235,0
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550,8,4,3,...,25,4,7,2019,3,1,17.0,44,3.625000,0
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297,29,1,6,...,29,2,12,2020,4,1,6.0,0,0.000000,0
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888,9,2,6,...,9,3,9,2020,3,1,3.0,0,0.000000,0
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915,10,3,9,...,10,7,1,2021,1,0,76.0,917,190.486842,0


In [15]:
user_dummies = pd.get_dummies(user_df['credit_type'], drop_first=True, prefix="c", dtype=int)
user_df = pd.concat([user_df, user_dummies], axis=1)

user_dummies_t = pd.get_dummies(user_df_t['credit_type'], drop_first=True, prefix="c", dtype=int)
user_df_t = pd.concat([user_df_t, user_dummies_t], axis=1)

user_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,o_date,o_day,o_month,...,c_Interbank credit,c_Loan for business development,c_Loan for purchase of shares (margin lending),c_Loan for the purchase of equipment,c_Loan for working capital replenishment,c_Microloan,c_Mobile operator loan,c_Mortgage,c_Real estate loan,c_Unknown type of loan
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,"[0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0,...",AAA09044550,22,6,9,...,0,0,0,0,0,0,0,0,0,0
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,"[0, 0, 0, 0, 0, 14, 44, 0, 0, 0, 0, 0, 0, 0, 0...",AAA09044550,8,4,3,...,0,0,0,0,0,0,0,0,0,0
2,Credit card,80996.445,0.0,2020-06-29,2020-12-29,"[0, 0, 0, 0, 0, 0]",AAA10545297,29,1,6,...,0,0,0,0,0,0,0,0,0,0
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,"[0, 0, 0]",AAA14112888,9,2,6,...,0,0,0,0,0,0,0,0,0,0
4,Credit card,10480.500,0.0,2014-09-10,2021-01-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAA20326915,10,3,9,...,0,0,0,0,0,0,0,0,0,0


In [16]:
agg_funcs = {
	'loan_amount': ['count', 'sum', 'mean', 'max', 'min'],
    'amount_overdue': ['sum', 'mean', 'max', 'min'],
    'open_date': ['max', 'min'],
    'closed_date': ['max', 'min'],
	'max_delay': ['sum', 'max'],
	'avg_delay': ['sum', 'max', 'min'],
	'loan_duration_m': ['mean', 'max', 'min'],
	'is_loan_active': ['max'],
	'credit_type': pd.Series.nunique
}

agg_numerics = user_df.groupby('uid').agg(agg_funcs).reset_index()
agg_numerics_t = user_df_t.groupby('uid').agg(agg_funcs).reset_index()


agg_numerics.head()

uid loan_amount                                                  \
                     count          sum        mean         max        min   
0  AAA09044550           2   277245.000  138622.500  272745.000   4500.000   
1  AAA10545297           1    80996.445   80996.445   80996.445  80996.445   
2  AAA14112888           1    43771.500   43771.500   43771.500  43771.500   
3  AAA20326915           8   591597.000   73949.625  235800.000      0.000   
4  AAA31604840           5  1591960.500  318392.100  687150.000  41845.500   

  amount_overdue                 ... max_delay        avg_delay              \
             sum mean  max  min  ...       sum  max         sum         max   
0            0.0  0.0  0.0  0.0  ...        54   44    4.213235    3.625000   
1            0.0  0.0  0.0  0.0  ...         0    0    0.000000    0.000000   
2            0.0  0.0  0.0  0.0  ...         0    0    0.000000    0.000000   
3            0.0  0.0  0.0  0.0  ...       917  917  190.486842  190.486842   
4            0.0  0.0  0.0  0.0  ...         0    0    0.000000    0.000000   

            loan_duration_m             is_loan_active credit_type  
        min            mean   max   min            max     nunique  
0  0.588235            17.0  17.0  17.0              0           1  
1  0.000000             6.0   6.0   6.0              0           1  
2  0.000000             3.0   3.0   3.0              0           1  
3  0.000000            19.0  76.0   1.0              0           2  
4  0.000000            15.6  43.0   2.0              0           2  

[5 rows x 24 columns]

In [17]:
agg_other = user_df.groupby('uid')[['o_date', 'o_day', 'o_month', 'o_year', 'o_quarter', 'o_is_weekday', 'c_date', \
									 'c_day', 'c_month', 'c_year', 'c_quarter', 'c_is_weekday']].agg(['first', 'last']).reset_index()
agg_other_t = user_df_t.groupby('uid')[['o_date', 'o_day', 'o_month', 'o_year', 'o_quarter', 'o_is_weekday', 'c_date', \
									 'c_day', 'c_month', 'c_year', 'c_quarter', 'c_is_weekday']].agg(['first', 'last']).reset_index()
agg_other.head()

uid o_date      o_day      o_month      o_year       o_quarter  \
                first last first last   first last  first  last     first   
0  AAA09044550     22    8     6    4       9    3   2018  2018         3   
1  AAA10545297     29   29     1    1       6    6   2020  2020         2   
2  AAA14112888      9    9     2    2       6    6   2020  2020         2   
3  AAA20326915     10   28     3    7       9    6   2014  2020         3   
4  AAA31604840     10   16     6    3      10    9   2020  2020         4   

   ... c_day      c_month      c_year       c_quarter      c_is_weekday       
   ... first last   first last  first  last     first last        first last  
0  ...     6    4       2    7   2020  2019         1    3            0    1  
1  ...     2    2      12   12   2020  2020         4    4            1    1  
2  ...     3    3       9    9   2020  2020         3    3            1    1  
3  ...     7    1       1   12   2021  2020         1    4            0    1  
4  ...     4    3      12   12   2020  2020         4    4            1    1  

[5 rows x 25 columns]

In [18]:
agg_binary = user_df.groupby('uid')[user_dummies.columns].agg(['max']).reset_index()
agg_binary_t = user_df_t.groupby('uid')[user_dummies_t.columns].agg(['max']).reset_index()

In [19]:
agg_binary.head()

,uid,c_Car loan,c_Cash loan (non-earmarked),c_Consumer credit,c_Credit card,c_Interbank credit,c_Loan for business development,c_Loan for purchase of shares (margin lending),c_Loan for the purchase of equipment,c_Loan for working capital replenishment,c_Microloan,c_Mobile operator loan,c_Mortgage,c_Real estate loan,c_Unknown type of loan
,,max,max,max,max,max,max,max,max,max,max,max,max,max,max
0,AAA09044550,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,AAA10545297,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,AAA14112888,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,AAA20326915,0,0,1,1,0,0,0,0,0,0,0,0,0,0
4,AAA31604840,0,0,1,1,0,0,0,0,0,0,0,0,0,0


In [20]:
agg_numerics.columns = ['_'.join(col).strip() for col in agg_numerics.columns]
agg_other.columns = ['_'.join(col).strip() for col in agg_other.columns]
agg_binary.columns = ['_'.join(col).strip() for col in agg_binary.columns]

agg_numerics_t.columns = ['_'.join(col).strip() for col in agg_numerics_t.columns]
agg_other_t.columns = ['_'.join(col).strip() for col in agg_other_t.columns]
agg_binary_t.columns = ['_'.join(col).strip() for col in agg_binary_t.columns]
agg_numerics.head()

,uid_,loan_amount_count,loan_amount_sum,loan_amount_mean,loan_amount_max,loan_amount_min,amount_overdue_sum,amount_overdue_mean,amount_overdue_max,amount_overdue_min,...,max_delay_sum,max_delay_max,avg_delay_sum,avg_delay_max,avg_delay_min,loan_duration_m_mean,loan_duration_m_max,loan_duration_m_min,is_loan_active_max,credit_type_nunique
0,AAA09044550,2,277245.000,138622.500,272745.000,4500.000,0.0,0.0,0.0,0.0,...,54,44,4.213235,3.625000,0.588235,17.0,17.0,17.0,0,1
1,AAA10545297,1,80996.445,80996.445,80996.445,80996.445,0.0,0.0,0.0,0.0,...,0,0,0.000000,0.000000,0.000000,6.0,6.0,6.0,0,1
2,AAA14112888,1,43771.500,43771.500,43771.500,43771.500,0.0,0.0,0.0,0.0,...,0,0,0.000000,0.000000,0.000000,3.0,3.0,3.0,0,1
3,AAA20326915,8,591597.000,73949.625,235800.000,0.000,0.0,0.0,0.0,0.0,...,917,917,190.486842,190.486842,0.000000,19.0,76.0,1.0,0,2
4,AAA31604840,5,1591960.500,318392.100,687150.000,41845.500,0.0,0.0,0.0,0.0,...,0,0,0.000000,0.000000,0.000000,15.6,43.0,2.0,0,2


In [21]:
agg_other.head()

,uid_,o_date_first,o_date_last,o_day_first,o_day_last,o_month_first,o_month_last,o_year_first,o_year_last,o_quarter_first,...,c_day_first,c_day_last,c_month_first,c_month_last,c_year_first,c_year_last,c_quarter_first,c_quarter_last,c_is_weekday_first,c_is_weekday_last
0,AAA09044550,22,8,6,4,9,3,2018,2018,3,...,6,4,2,7,2020,2019,1,3,0,1
1,AAA10545297,29,29,1,1,6,6,2020,2020,2,...,2,2,12,12,2020,2020,4,4,1,1
2,AAA14112888,9,9,2,2,6,6,2020,2020,2,...,3,3,9,9,2020,2020,3,3,1,1
3,AAA20326915,10,28,3,7,9,6,2014,2020,3,...,7,1,1,12,2021,2020,1,4,0,1
4,AAA31604840,10,16,6,3,10,9,2020,2020,4,...,4,3,12,12,2020,2020,4,4,1,1


In [22]:
agg_binary.head()

,uid_,c_Car loan_max,c_Cash loan (non-earmarked)_max,c_Consumer credit_max,c_Credit card_max,c_Interbank credit_max,c_Loan for business development_max,c_Loan for purchase of shares (margin lending)_max,c_Loan for the purchase of equipment_max,c_Loan for working capital replenishment_max,c_Microloan_max,c_Mobile operator loan_max,c_Mortgage_max,c_Real estate loan_max,c_Unknown type of loan_max
0,AAA09044550,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,AAA10545297,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,AAA14112888,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,AAA20326915,0,0,1,1,0,0,0,0,0,0,0,0,0,0
4,AAA31604840,0,0,1,1,0,0,0,0,0,0,0,0,0,0


In [23]:
user_df = agg_numerics.merge(agg_other, on='uid_', how='outer').merge(agg_binary, on='uid_', how='outer')

user_df_t = agg_numerics_t.merge(agg_other_t, on='uid_', how='outer').merge(agg_binary_t, on='uid_', how='outer')


user_df.head()

,uid_,loan_amount_count,loan_amount_sum,loan_amount_mean,loan_amount_max,loan_amount_min,amount_overdue_sum,amount_overdue_mean,amount_overdue_max,amount_overdue_min,...,c_Interbank credit_max,c_Loan for business development_max,c_Loan for purchase of shares (margin lending)_max,c_Loan for the purchase of equipment_max,c_Loan for working capital replenishment_max,c_Microloan_max,c_Mobile operator loan_max,c_Mortgage_max,c_Real estate loan_max,c_Unknown type of loan_max
0,AAA09044550,2,277245.000,138622.500,272745.000,4500.000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,AAA10545297,1,80996.445,80996.445,80996.445,80996.445,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,AAA14112888,1,43771.500,43771.500,43771.500,43771.500,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,AAA20326915,8,591597.000,73949.625,235800.000,0.000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,AAA31604840,5,1591960.500,318392.100,687150.000,41845.500,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# (user_df.columns == user_df_t.columns).all()
# user_df.shape, user_df_t.shape
# set(user_df.columns) - set(user_df_t.columns)
set(user_df.columns) == set(user_df_t.columns)

False

In [25]:
user_df_t[['c_Interbank credit_max', 'c_Loan for purchase of shares (margin lending)_max', 'c_Mobile operator loan_max']] = 0
user_df_t.head()

,uid_,loan_amount_count,loan_amount_sum,loan_amount_mean,loan_amount_max,loan_amount_min,amount_overdue_sum,amount_overdue_mean,amount_overdue_max,amount_overdue_min,...,c_Loan for business development_max,c_Loan for the purchase of equipment_max,c_Loan for working capital replenishment_max,c_Microloan_max,c_Mortgage_max,c_Real estate loan_max,c_Unknown type of loan_max,c_Interbank credit_max,c_Loan for purchase of shares (margin lending)_max,c_Mobile operator loan_max
0,AAA14437029,12,3736315.890,311359.657500,1575000.0,14613.39,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,AAB12915377,3,408007.980,136002.660000,247500.0,45532.98,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,AAB55088883,6,327394.485,54565.747500,114471.9,4500.00,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,AAB68152393,7,1806736.500,258105.214286,711000.0,63049.50,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,AAC29580834,5,2867536.800,573507.360000,1422000.0,40860.00,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
user_df[user_df.isna().any(axis=1)]

,uid_,loan_amount_count,loan_amount_sum,loan_amount_mean,loan_amount_max,loan_amount_min,amount_overdue_sum,amount_overdue_mean,amount_overdue_max,amount_overdue_min,...,c_Interbank credit_max,c_Loan for business development_max,c_Loan for purchase of shares (margin lending)_max,c_Loan for the purchase of equipment_max,c_Loan for working capital replenishment_max,c_Microloan_max,c_Mobile operator loan_max,c_Mortgage_max,c_Real estate loan_max,c_Unknown type of loan_max
26705,DCX71532515,0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
user_df.columns

Index(['uid_', 'loan_amount_count', 'loan_amount_sum', 'loan_amount_mean',
       'loan_amount_max', 'loan_amount_min', 'amount_overdue_sum',
       'amount_overdue_mean', 'amount_overdue_max', 'amount_overdue_min',
       'open_date_max', 'open_date_min', 'closed_date_max', 'closed_date_min',
       'max_delay_sum', 'max_delay_max', 'avg_delay_sum', 'avg_delay_max',
       'avg_delay_min', 'loan_duration_m_mean', 'loan_duration_m_max',
       'loan_duration_m_min', 'is_loan_active_max', 'credit_type_nunique',
       'o_date_first', 'o_date_last', 'o_day_first', 'o_day_last',
       'o_month_first', 'o_month_last', 'o_year_first', 'o_year_last',
       'o_quarter_first', 'o_quarter_last', 'o_is_weekday_first',
       'o_is_weekday_last', 'c_date_first', 'c_date_last', 'c_day_first',
       'c_day_last', 'c_month_first', 'c_month_last', 'c_year_first',
       'c_year_last', 'c_quarter_first', 'c_quarter_last',
       'c_is_weekday_first', 'c_is_weekday_last', 'c_Car loan_max',
       'c

In [28]:
enq_data.head()

,enquiry_type,enquiry_amt,enquiry_date,uid
0,Interbank credit,168839,2020-11-08,AAA08065248
1,Mobile operator loan,268392,2020-09-20,AAA08065248
2,Mobile operator loan,36082,2020-06-19,AAA08065248
3,Interbank credit,180467,2019-10-22,AAA08065248
4,Cash loan (non-earmarked),227459,2020-05-24,AAA08065248


<h3>Enquiry - Date Based Features</h3>

In [29]:
enq_df = enq_data.copy()
enq_df_t = enq_data_t.copy()

enq_df['enquiry_date'] = pd.to_datetime(enq_df['enquiry_date'])
enq_df = extract_date_features(enq_df, column='enquiry_date')

enq_df_t['enquiry_date'] = pd.to_datetime(enq_df_t['enquiry_date'])
enq_df_t = extract_date_features(enq_df_t, column='enquiry_date')
enq_df.head()

,enquiry_type,enquiry_amt,enquiry_date,uid,e_date,e_day,e_month,e_year,e_quarter,e_is_weekday
0,Interbank credit,168839,2020-11-08,AAA08065248,8,7,11,2020,4,False
1,Mobile operator loan,268392,2020-09-20,AAA08065248,20,7,9,2020,3,False
2,Mobile operator loan,36082,2020-06-19,AAA08065248,19,5,6,2020,2,True
3,Interbank credit,180467,2019-10-22,AAA08065248,22,2,10,2019,4,True
4,Cash loan (non-earmarked),227459,2020-05-24,AAA08065248,24,7,5,2020,2,False


In [30]:
enq_df[enq_df.select_dtypes(bool).columns] = enq_df.select_dtypes(bool).astype(int)

enq_df_t[enq_df_t.select_dtypes(bool).columns] = enq_df_t.select_dtypes(bool).astype(int)
enq_df.head()

,enquiry_type,enquiry_amt,enquiry_date,uid,e_date,e_day,e_month,e_year,e_quarter,e_is_weekday
0,Interbank credit,168839,2020-11-08,AAA08065248,8,7,11,2020,4,0
1,Mobile operator loan,268392,2020-09-20,AAA08065248,20,7,9,2020,3,0
2,Mobile operator loan,36082,2020-06-19,AAA08065248,19,5,6,2020,2,1
3,Interbank credit,180467,2019-10-22,AAA08065248,22,2,10,2019,4,1
4,Cash loan (non-earmarked),227459,2020-05-24,AAA08065248,24,7,5,2020,2,0


<h3>Enquiry - Numerical Features</h3>

In [31]:
enq_dummies = pd.get_dummies(enq_df['enquiry_type'], drop_first=True, prefix="e", dtype=int)
enq_df = pd.concat([enq_df, enq_dummies], axis=1)

enq_dummies_t = pd.get_dummies(enq_df_t['enquiry_type'], drop_first=True, prefix="e", dtype=int)
enq_df_t = pd.concat([enq_df_t, enq_dummies_t], axis=1)

enq_df.head()

,enquiry_type,enquiry_amt,enquiry_date,uid,e_date,e_day,e_month,e_year,e_quarter,e_is_weekday,...,e_Loan for business development,e_Loan for purchase of shares (margin lending),e_Loan for the purchase of equipment,e_Loan for working capital replenishment,e_Microloan,e_Mobile operator loan,e_Mortgage,e_Real estate loan,e_Revolving loans,e_Unknown type of loan
0,Interbank credit,168839,2020-11-08,AAA08065248,8,7,11,2020,4,0,...,0,0,0,0,0,0,0,0,0,0
1,Mobile operator loan,268392,2020-09-20,AAA08065248,20,7,9,2020,3,0,...,0,0,0,0,0,1,0,0,0,0
2,Mobile operator loan,36082,2020-06-19,AAA08065248,19,5,6,2020,2,1,...,0,0,0,0,0,1,0,0,0,0
3,Interbank credit,180467,2019-10-22,AAA08065248,22,2,10,2019,4,1,...,0,0,0,0,0,0,0,0,0,0
4,Cash loan (non-earmarked),227459,2020-05-24,AAA08065248,24,7,5,2020,2,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
agg_funcs = {
	'enquiry_amt': ['count', 'sum', 'mean', 'max', 'min'],
	'enquiry_date':['max', 'min'],
	'enquiry_type': pd.Series.nunique
}

agg_numerics = enq_df.groupby('uid').agg(agg_funcs).reset_index()

agg_numerics_t = enq_df_t.groupby('uid').agg(agg_funcs).reset_index()

agg_numerics.head()

uid enquiry_amt                                         \
                     count      sum           mean     max    min   
0  AAA08065248          11  2064658  187696.181818  364751  36082   
1  AAA09044550          26  2659000  102269.230769  197000   5000   
2  AAA10545297          14  1317000   94071.428571  192000   5000   
3  AAA14112888          15  1465000   97666.666667  185000  17000   
4  AAA20326915           1    66000   66000.000000   66000  66000   

  enquiry_date            enquiry_type  
           max        min      nunique  
0   2020-12-30 2019-03-01            6  
1   2020-12-29 2017-10-17           14  
2   2020-10-29 2018-07-06            9  
3   2020-07-05 2017-04-13           10  
4   2020-08-14 2020-08-14            1

In [33]:
agg_other = enq_df.groupby('uid')[['e_date', 'e_day', 'e_month', 'e_year', 'e_quarter', 'e_is_weekday']].agg(['first', 'last']).reset_index()

agg_other_t = enq_df_t.groupby('uid')[['e_date', 'e_day', 'e_month', 'e_year', 'e_quarter', 'e_is_weekday']].agg(['first', 'last']).reset_index()
agg_other.head()

uid e_date      e_day      e_month      e_year       e_quarter  \
                first last first last   first last  first  last     first   
0  AAA08065248      8   30     7    3      11   12   2020  2020         4   
1  AAA09044550     19   12     1    7       8    5   2019  2019         3   
2  AAA10545297     22    6     1    5       4    7   2019  2018         2   
3  AAA14112888      9   10     6    7      11   11   2019  2019         4   
4  AAA20326915     14   14     5    5       8    8   2020  2020         3   

       e_is_weekday       
  last        first last  
0    4            0    1  
1    2            1    0  
2    3            1    1  
3    4            0    0  
4    3            1    1

In [34]:
agg_numerics.columns

MultiIndex([(         'uid',        ''),
            ( 'enquiry_amt',   'count'),
            ( 'enquiry_amt',     'sum'),
            ( 'enquiry_amt',    'mean'),
            ( 'enquiry_amt',     'max'),
            ( 'enquiry_amt',     'min'),
            ('enquiry_date',     'max'),
            ('enquiry_date',     'min'),
            ('enquiry_type', 'nunique')],
           )

In [35]:
agg_binary = enq_df.groupby('uid')[enq_dummies.columns].agg(['max']).reset_index()
agg_binary_t = enq_df_t.groupby('uid')[enq_dummies_t.columns].agg(['max']).reset_index()
agg_binary.head()

,uid,e_Car loan,e_Cash loan (non-earmarked),e_Cash loans,e_Consumer credit,e_Credit card,e_Interbank credit,e_Loan for business development,e_Loan for purchase of shares (margin lending),e_Loan for the purchase of equipment,e_Loan for working capital replenishment,e_Microloan,e_Mobile operator loan,e_Mortgage,e_Real estate loan,e_Revolving loans,e_Unknown type of loan
,,max,max,max,max,max,max,max,max,max,max,max,max,max,max,max,max
0,AAA08065248,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,0
1,AAA09044550,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1
2,AAA10545297,0,1,1,1,1,1,0,1,0,0,0,0,0,0,1,1
3,AAA14112888,0,1,1,1,0,1,0,1,1,1,0,1,0,0,0,1
4,AAA20326915,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [36]:
# agg_numerics.head()
# agg_other.head()
# agg_binary.head()

In [37]:
agg_numerics.columns = ['_'.join(col).strip() for col in agg_numerics.columns]
agg_other.columns = ['_'.join(col).strip() for col in agg_other.columns]
agg_binary.columns = ['_'.join(col).strip() for col in agg_binary.columns]
enq_df = agg_numerics.merge(agg_other, on='uid_', how='inner').merge(agg_binary, on='uid_', how='inner')

agg_numerics_t.columns = ['_'.join(col).strip() for col in agg_numerics_t.columns]
agg_other_t.columns = ['_'.join(col).strip() for col in agg_other_t.columns]
agg_binary_t.columns = ['_'.join(col).strip() for col in agg_binary_t.columns]
enq_df_t = agg_numerics_t.merge(agg_other_t, on='uid_', how='inner').merge(agg_binary_t, on='uid_', how='inner')

enq_df.head()

,uid_,enquiry_amt_count,enquiry_amt_sum,enquiry_amt_mean,enquiry_amt_max,enquiry_amt_min,enquiry_date_max,enquiry_date_min,enquiry_type_nunique,e_date_first,...,e_Loan for business development_max,e_Loan for purchase of shares (margin lending)_max,e_Loan for the purchase of equipment_max,e_Loan for working capital replenishment_max,e_Microloan_max,e_Mobile operator loan_max,e_Mortgage_max,e_Real estate loan_max,e_Revolving loans_max,e_Unknown type of loan_max
0,AAA08065248,11,2064658,187696.181818,364751,36082,2020-12-30,2019-03-01,6,8,...,0,1,0,0,0,1,1,0,1,0
1,AAA09044550,26,2659000,102269.230769,197000,5000,2020-12-29,2017-10-17,14,19,...,0,0,1,0,1,1,1,1,1,1
2,AAA10545297,14,1317000,94071.428571,192000,5000,2020-10-29,2018-07-06,9,22,...,0,1,0,0,0,0,0,0,1,1
3,AAA14112888,15,1465000,97666.666667,185000,17000,2020-07-05,2017-04-13,10,9,...,0,1,1,1,0,1,0,0,0,1
4,AAA20326915,1,66000,66000.000000,66000,66000,2020-08-14,2020-08-14,1,14,...,0,0,0,0,0,1,0,0,0,0


In [38]:
# set(enq_df.columns) - set(enq_df_t.columns)
set(enq_df.columns) == set(enq_df_t.columns)

True

In [39]:
user_df.shape, enq_df.shape, user_df_t.shape, enq_df_t.shape

((223918, 62), (261383, 37), (39572, 62), (46127, 37))

In [40]:
missing_in_enq = user_df[~user_df['uid_'].isin(enq_df['uid_'])]
missing_in_user = enq_df[~enq_df['uid_'].isin(user_df['uid_'])]
# missing_in_df1.head()
# user_df['uid_'].head()
missing_in_enq.shape, missing_in_user.shape

((0, 62), (37465, 37))

<h1>Data Aggregation</h2>

In [41]:
combined_df = user_df.merge(enq_df, on='uid_', how='outer').merge(train_ops, left_on='uid_', right_on='uid', how='outer')
combined_df_t = user_df_t.merge(enq_df_t, on='uid_', how='outer').merge(test_ops, left_on='uid_', right_on='uid', how='outer')
combined_df.shape, combined_df_t.shape

((261383, 101), (46127, 100))

In [42]:
combined_df.shape, combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261383 entries, 0 to 261382
Columns: 101 entries, uid_ to TARGET
dtypes: datetime64[ns](6), float64(58), int32(10), int64(24), object(3)
memory usage: 191.4+ MB


((261383, 101), None)

In [43]:
missed = pd.DataFrame(combined_df.isna().sum().items(), columns=['Name', 'Count'])
missed[missed['Count']>0]

,Name,Count
1,loan_amount_count,37465
2,loan_amount_sum,37465
3,loan_amount_mean,37466
4,loan_amount_max,37466
5,loan_amount_min,37466
...,...,...
57,c_Microloan_max,37465
58,c_Mobile operator loan_max,37465
59,c_Mortgage_max,37465
60,c_Real estate loan_max,37465


In [44]:
print(combined_df.columns)

Index(['uid_', 'loan_amount_count', 'loan_amount_sum', 'loan_amount_mean',
       'loan_amount_max', 'loan_amount_min', 'amount_overdue_sum',
       'amount_overdue_mean', 'amount_overdue_max', 'amount_overdue_min',
       ...
       'e_Loan for working capital replenishment_max', 'e_Microloan_max',
       'e_Mobile operator loan_max', 'e_Mortgage_max',
       'e_Real estate loan_max', 'e_Revolving loans_max',
       'e_Unknown type of loan_max', 'uid', 'NAME_CONTRACT_TYPE', 'TARGET'],
      dtype='object', length=101)


In [45]:
binary_columns = combined_df.columns[combined_df.nunique() == 2]
binary_columns

Index(['is_loan_active_max', 'o_is_weekday_first', 'o_is_weekday_last',
       'c_is_weekday_first', 'c_is_weekday_last', 'c_Car loan_max',
       'c_Cash loan (non-earmarked)_max', 'c_Consumer credit_max',
       'c_Credit card_max', 'c_Interbank credit_max',
       'c_Loan for business development_max',
       'c_Loan for purchase of shares (margin lending)_max',
       'c_Loan for the purchase of equipment_max',
       'c_Loan for working capital replenishment_max', 'c_Microloan_max',
       'c_Mobile operator loan_max', 'c_Mortgage_max',
       'c_Real estate loan_max', 'c_Unknown type of loan_max',
       'e_is_weekday_first', 'e_is_weekday_last', 'e_Car loan_max',
       'e_Cash loan (non-earmarked)_max', 'e_Cash loans_max',
       'e_Consumer credit_max', 'e_Credit card_max', 'e_Interbank credit_max',
       'e_Loan for business development_max',
       'e_Loan for purchase of shares (margin lending)_max',
       'e_Loan for the purchase of equipment_max',
       'e_Loan for wor

In [46]:
combined_df[binary_columns[:-1]] = combined_df[binary_columns[:-1]].fillna(0)
combined_df_t[binary_columns[:-1]] = combined_df_t[binary_columns[:-1]].fillna(0)

In [47]:
missed = pd.DataFrame(combined_df.isna().sum().items(), columns=['Name', 'Count'])
missed[missed['Count']>0]

,Name,Count
1,loan_amount_count,37465
2,loan_amount_sum,37465
3,loan_amount_mean,37466
4,loan_amount_max,37466
5,loan_amount_min,37466
6,amount_overdue_sum,37465
7,amount_overdue_mean,37465
8,amount_overdue_max,37465
9,amount_overdue_min,37465
10,open_date_max,37465


In [48]:
numeric_columns = combined_df.select_dtypes('number').columns
numeric_columns_t = combined_df_t.select_dtypes('number').columns
# numeric_columns
combined_df[numeric_columns] = combined_df[numeric_columns].fillna(combined_df[numeric_columns].mean())

combined_df_t[numeric_columns_t] = combined_df_t[numeric_columns_t].fillna(combined_df_t[numeric_columns_t].mean())
combined_df.shape, combined_df_t.shape

((261383, 101), (46127, 100))

In [49]:
# type(combined_df['credit_type_unique'].iloc[1]) == np.ndarray
z = np.array('')
type(z)

numpy.ndarray

In [50]:
# combined_df['credit_type_unique'] = combined_df['credit_type_unique'].fillna('').apply(lambda x: x if type(x)==np.ndarray else np.array(['']))
# combined_df_t['credit_type_unique'] = combined_df_t['credit_type_unique'].fillna('').apply(lambda x: x if type(x)==np.ndarray else np.array(['']))

combined_df.head()

,uid_,loan_amount_count,loan_amount_sum,loan_amount_mean,loan_amount_max,loan_amount_min,amount_overdue_sum,amount_overdue_mean,amount_overdue_max,amount_overdue_min,...,e_Loan for working capital replenishment_max,e_Microloan_max,e_Mobile operator loan_max,e_Mortgage_max,e_Real estate loan_max,e_Revolving loans_max,e_Unknown type of loan_max,uid,NAME_CONTRACT_TYPE,TARGET
0,AAA08065248,5.561442,1.959597e+06,379285.495581,977631.738134,119722.433128,233.708386,54.832291,198.1287,14.999448,...,0,0,1,1,0,1,0,AAA08065248,Revolving loans,0
1,AAA09044550,2.000000,2.772450e+05,138622.500000,272745.000000,4500.000000,0.000000,0.000000,0.0000,0.000000,...,0,1,1,1,1,1,1,AAA09044550,Cash loans,0
2,AAA10545297,1.000000,8.099645e+04,80996.445000,80996.445000,80996.445000,0.000000,0.000000,0.0000,0.000000,...,0,0,0,0,0,1,1,AAA10545297,Cash loans,0
3,AAA14112888,1.000000,4.377150e+04,43771.500000,43771.500000,43771.500000,0.000000,0.000000,0.0000,0.000000,...,1,0,1,0,0,0,1,AAA14112888,Cash loans,0
4,AAA20326915,8.000000,5.915970e+05,73949.625000,235800.000000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0,0,1,0,0,0,0,AAA20326915,Cash loans,0


In [51]:
# type(combined_df['credit_type_unique'].iloc[0])

In [52]:
# combined_df['credit_type_unique'].head()

In [53]:
missed = pd.DataFrame(combined_df.isna().sum().items(), columns=['Name', 'Count'])
missed[missed['Count']>0]

,Name,Count
10,open_date_max,37465
11,open_date_min,37465
12,closed_date_max,37465
13,closed_date_min,37465


In [54]:
missed = pd.DataFrame(combined_df_t.isna().sum().items(), columns=['Name', 'Count'])
missed[missed['Count']>0]

,Name,Count
10,open_date_max,6555
11,open_date_min,6555
12,closed_date_max,6555
13,closed_date_min,6555


In [55]:
# types = pd.DataFrame(combined_df.dtypes.items(), columns=['Name', 'Type'])
# types[(types['Type'] == "<M8[ns]") | (types['Type'] =="object")]

types = pd.DataFrame(combined_df.select_dtypes(exclude='number').items(), columns=['Name', 'Type'])

types

,Name,Type
0,uid_,0 AAA08065248 1 AAA09044550 2 ...
1,open_date_max,0 NaT 1 2018-09-22 2 ...
2,open_date_min,0 NaT 1 2018-03-08 2 ...
3,closed_date_max,0 NaT 1 2020-02-22 2 ...
4,closed_date_min,0 NaT 1 2019-07-25 2 ...
5,enquiry_date_max,0 2020-12-30 1 2020-12-29 2 ...
6,enquiry_date_min,0 2019-03-01 1 2017-10-17 2 ...
7,uid,0 AAA08065248 1 AAA09044550 2 ...
8,NAME_CONTRACT_TYPE,0 Revolving loans 1 Cash ...


In [56]:
# combined_df['NAME_CONTRACT_TYPE'] = combined_df['NAME_CONTRACT_TYPE'].apply(lambda x: 1 if x=='Revolving loans' else 0)
# combined_df_t['NAME_CONTRACT_TYPE'] = combined_df_t['NAME_CONTRACT_TYPE'].apply(lambda x: 1 if x=='Revolving loans' else 0)

combined_df['NAME_CONTRACT_TYPE'].value_counts()

NAME_CONTRACT_TYPE
Cash loans         236524
Revolving loans     24859
Name: count, dtype: int64

<h1>Model Training</h1>

In [57]:
print([x for x in combined_df.select_dtypes('number').columns])

['loan_amount_count', 'loan_amount_sum', 'loan_amount_mean', 'loan_amount_max', 'loan_amount_min', 'amount_overdue_sum', 'amount_overdue_mean', 'amount_overdue_max', 'amount_overdue_min', 'max_delay_sum', 'max_delay_max', 'avg_delay_sum', 'avg_delay_max', 'avg_delay_min', 'loan_duration_m_mean', 'loan_duration_m_max', 'loan_duration_m_min', 'is_loan_active_max', 'credit_type_nunique', 'o_date_first', 'o_date_last', 'o_day_first', 'o_day_last', 'o_month_first', 'o_month_last', 'o_year_first', 'o_year_last', 'o_quarter_first', 'o_quarter_last', 'o_is_weekday_first', 'o_is_weekday_last', 'c_date_first', 'c_date_last', 'c_day_first', 'c_day_last', 'c_month_first', 'c_month_last', 'c_year_first', 'c_year_last', 'c_quarter_first', 'c_quarter_last', 'c_is_weekday_first', 'c_is_weekday_last', 'c_Car loan_max', 'c_Cash loan (non-earmarked)_max', 'c_Consumer credit_max', 'c_Credit card_max', 'c_Interbank credit_max', 'c_Loan for business development_max', 'c_Loan for purchase of shares (margin l

In [58]:
# final_features = ['loan_amount_count', 'loan_amount_sum', 'loan_amount_mean',
#        'loan_amount_max', 'loan_amount_min', 'amount_overdue_sum',
#        'amount_overdue_mean', 'amount_overdue_max', 'amount_overdue_min',
#        'max_delay_sum', 'max_delay_max', 'avg_delay_sum', 'avg_delay_max',
#        'avg_delay_min', 'loan_duration_m_mean', 'loan_duration_m_max',
#        'loan_duration_m_min', 'is_loan_active_max',
#        'o_date_first', 'o_date_last', 'o_day_first', 'o_day_last',
#        'o_month_first', 'o_month_last', 'o_year_first', 'o_year_last',
#        'o_quarter_first', 'o_quarter_last', 'o_is_weekday_first',
#        'o_is_weekday_last', 'c_date_first', 'c_date_last', 'c_day_first',
#        'c_day_last', 'c_month_first', 'c_month_last', 'c_year_first',
#        'c_year_last', 'c_quarter_first', 'c_quarter_last',
#        'c_is_weekday_first', 'c_is_weekday_last', 'enquiry_amt_count',
#        'enquiry_amt_sum', 'enquiry_amt_mean', 'enquiry_amt_max',
#        'enquiry_amt_min', 'e_date_first', 'e_date_last', 'e_day_first',
#        'e_day_last', 'e_month_first', 'e_month_last', 'e_year_first',
#        'e_year_last', 'e_quarter_first', 'e_quarter_last',
#        'e_is_weekday_first', 'e_is_weekday_last', 'TARGET']

final_features = combined_df.select_dtypes('number').columns

final_data = combined_df[final_features]
final_data.dtypes

loan_amount_count             float64
loan_amount_sum               float64
loan_amount_mean              float64
loan_amount_max               float64
loan_amount_min               float64
                               ...   
e_Mortgage_max                  int64
e_Real estate loan_max          int64
e_Revolving loans_max           int64
e_Unknown type of loan_max      int64
TARGET                          int64
Length: 92, dtype: object

In [59]:
X = final_data.drop(columns=['TARGET'])
y = final_data['TARGET']

In [60]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.15, random_state=42)

In [61]:
y_train.value_counts()

TARGET
0    204277
1     17898
Name: count, dtype: int64

In [62]:
# train_df = pd.concat([X_train, y_train], axis=1)
# train_majority = train_df[train_df['TARGET'] == 0]
# train_minority = train_df[train_df['TARGET'] == 1]

# # Resample Minor class
# train_minority_upsampled = train_minority.sample(n=len(train_majority), replace=True, random_state=42)

# # Resample both class
# # train_minority_upsampled = train_minority.sample(n=len(train_majority)//4, replace=True, random_state=42)
# # train_majority_downsampled = train_majority.sample(n=len(train_majority)//4, replace=True, random_state=42)

# # Combine
# # train_resampled = pd.concat([train_majority_downsampled, train_minority_upsampled])
# train_resampled = pd.concat([train_majority, train_minority_upsampled])

# # shuffle
# train_resampled = train_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [63]:
# X_train = train_resampled.drop(columns='TARGET')
# y_train = train_resampled['TARGET']
# y_train.value_counts()

In [64]:
# smote = SMOTE(random_state=42)
# X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# y_train_smote.value_counts()

In [65]:

# rf_model = RandomForestClassifier(n_estimators=30, min_samples_leaf=3, min_samples_split=4, class_weight='balanced', random_state=42)
# rf_model.fit(X_train, y_train)

# y_pred_rf = rf_model.predict(X_val)

# print(f"ROC_AUC Score for Basic XGB Model : { roc_auc_score(y_pred_rf, y_val)}")

In [66]:

kf = KFold(n_splits=5, shuffle=True, random_state=42)

xgbc = XGBClassifier(n_estimators=100, max_depth=4, min_child_weight=4, subsample=0.8, scale_pos_weight=0.1, eval_metric='auc', booster='gbtree')
xgbc.fit(X_train, y_train)

# cv_score = cross_val_score(xgbc, X_train, y_train, cv=kf, scoring='roc_auc', verbose=1)

# y_preds = xgbc.predict_proba(X_val)
# y_preds = np.array([0 if x>y else 1 for (x,y) in y_preds])

y_preds = xgbc.predict(X_val)

print(f"ROC_AUC Score for Basic XGB Model (model predictions) : { roc_auc_score(y_preds, y_val)}")
print("F1 Score:", f1_score(y_val, y_preds))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_preds))


ROC_AUC Score for Basic XGB Model (model predictions) : nan
F1 Score: 0.0
Confusion Matrix:
 [[36049     0]
 [ 3159     0]]


c:\Users\comp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:375: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [67]:
def find_best_thres(y_probs, y_val):
	best_thres = 0.01
	best_roc = 0.0

	for thresh in np.arange(0.00, 1.0, 0.001):
		preds = (y_probs >= thresh).astype(int)
		roc = roc_auc_score(y_val, preds)
		if roc > best_roc:
			best_roc = roc
			best_thres = thresh

	return best_thres, best_roc

In [68]:
y_probs = xgbc.predict_proba(X_val)[:, 1]  # Use your validation set

best_thres, best_roc = find_best_thres(y_probs, y_val)
print(f"Best threshold for ROC_AUC Score: {best_thres:.2f} → roc Score: {best_roc:.4f}")


Best threshold for ROC_AUC Score: 0.01 → roc Score: 0.5881


In [69]:
y_pred = (y_probs >= best_thres).astype(int)

print("F1 Score:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_probs))  # Note: use probs here
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

F1 Score: 0.18441899416051785
ROC AUC: 0.6263528298258805
Confusion Matrix:
 [[19154 16895]
 [ 1122  2037]]


In [70]:
param_grid = {
    'max_depth': [2, 3],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1],
	'min_child_weight':[2, 4],
	'scale_pos_weight': [0.35, 0.4]
}

In [71]:
# X_train, y_train, X_val, y_val = train_test_split(X_train, y_train, stratify=y, test_size=0.1, random_state=42)
# kf = KFold(n_splits=5, shuffle=True, random_state=42)


xgb_grid = XGBClassifier(eval_metric='auc', booster='gbtree')
grid_search = GridSearchCV(estimator=xgbc,
							param_grid=param_grid,
							scoring='roc_auc',
							cv=kf,
							verbose=1,
							n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster='gbtree',
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='auc', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=Non...
                                     max_delta_step=None, max_depth=4,
                                     max_leaves=None, min_child_weight=4,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=100,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3], 'min_child_weight': [2, 4],
                         'n_estimators': [100, 200],
                         'scale_pos_weight': [0.35, 0.4],
                         'subsample': [0.8, 1]},
             scoring='roc_auc', verbose=1)

In [72]:
grid_search.best_params_, grid_search.best_score_

({'max_depth': 2,
  'min_child_weight': 2,
  'n_estimators': 200,
  'scale_pos_weight': 0.35,
  'subsample': 1},
 np.float64(0.6334515223164983))

In [73]:
best_xgbc = grid_search.best_estimator_

y_xg_probs = best_xgbc.predict_proba(X_val)[:, 1]

best_thres, best_roc = find_best_thres(y_xg_probs, y_val)

# y_pred_xg = best_xgbc.predict(X_val)
print(best_thres, best_roc)

y_pred_xg = (y_xg_probs >= best_thres).astype(int)


print(f"ROC_AUC Score for Best Model : { roc_auc_score(y_val, y_xg_probs)}")
# type(y_pred_probs)


0.032 0.5903338357359273
ROC_AUC Score for Best Model : 0.6280856283414531


In [74]:
f1_score(y_val, y_pred_xg)

0.19215978878493944

In [75]:
importances = pd.Series(best_xgbc.feature_importances_, index=X_train.columns).sort_values(ascending=False)
importances.head(10)

amount_overdue_max     0.085913
amount_overdue_sum     0.082890
o_year_last            0.081084
c_Microloan_max        0.061326
o_year_first           0.052407
e_year_first           0.046347
e_year_last            0.036943
c_year_last            0.034490
amount_overdue_mean    0.034320
enquiry_amt_max        0.032520
dtype: float32

In [76]:
X_t = combined_df_t[final_features[:-1]]
# y_preds = xgbc.predict(X_t)
y_preds = xgbc.predict_proba(X_t)[:, 1]
y_preds = (y_preds >= best_thres).astype(int)

In [77]:
y_preds.sum()

np.int64(408)

In [80]:
op_df = pd.DataFrame({
	'uid': combined_df_t['uid'],
	'pred': y_preds
})

op_df.head(), op_df[op_df['pred'] == 1].shape

(           uid  pred
 0  AAA02107680     0
 1  AAA14437029     0
 2  AAB12915377     0
 3  AAB55088883     0
 4  AAB68152393     0,
 (408, 2))

In [81]:
op_df.to_csv('data/final_submission/Final_Probs_Nilesh_Mishra-lc1.csv', index=False)